In [88]:
import pandas as pd

df=pd.read_excel('../data/Food Delivery Time Prediction Case Study.xlsx')
df.drop(df[['ID','Delivery_person_ID','Type_of_order']],axis=1,inplace=True)

In [89]:
from geopy.distance import geodesic

def compute_distance(row):
    rest=(row['Restaurant_latitude'],row['Restaurant_longitude'])
    delv=(row['Delivery_location_latitude'],row['Delivery_location_longitude'])
    return geodesic(rest,delv).km
df['distance']=df.apply(compute_distance,axis=1)

In [90]:
df.drop(df[['Restaurant_longitude','Restaurant_latitude','Delivery_location_latitude','Delivery_location_longitude']],axis=1,inplace=True)

In [91]:
df=df[df['distance']<100]

In [92]:
df.head()

,Delivery_person_Age,Delivery_person_Ratings,Type_of_vehicle,Time_taken(min),distance
0,37,4.9,motorcycle,24,3.020737
1,34,4.5,scooter,33,20.143737
2,23,4.4,motorcycle,26,1.549693
3,38,4.7,motorcycle,21,7.774497
4,32,4.6,scooter,30,6.197898


In [93]:
df['Delivery_person_Ratings']=df['Delivery_person_Ratings'].clip(lower=2.5,upper=5)

In [94]:
speed={
    'motorcycle ':1,
    'scooter ':2,
    'electric_scooter ':3,
    'bicycle ':4
}
df['speed']=df['Type_of_vehicle'].map(speed)
df[['Type_of_vehicle','speed']]

,Type_of_vehicle,speed
0,motorcycle,1
1,scooter,2
2,motorcycle,1
3,motorcycle,1
4,scooter,2
...,...,...
45588,motorcycle,1
45589,motorcycle,1
45590,scooter,2
45591,motorcycle,1


In [95]:
df['capabilty']=df['Delivery_person_Ratings']*df['speed']
max_capability=df['Delivery_person_Ratings'].max()*df['speed'].max()
df['traffic_fact']=max_capability-df['capabilty']

In [96]:
from sklearn.preprocessing import LabelEncoder

la=LabelEncoder()
df['Type_of_vehicle']=la.fit_transform(df['Type_of_vehicle'])

In [97]:
df.to_csv('../data/cleaned.csv')